# 2024年度 データサイエンス実習 最終課題
## 23vr008n 三宅研 修士2年 高林 秀

# レポート説明

- 選択した問題：選択肢① プロ野球Freak
- 概要・注意事項
    - 「2009年から2023年までの⽇付や対戦相⼿など5-6の指標 (説明変数)を元にホームゲームの観客数を予測する回帰モデルを作り、重要な指標を理解する」
    - 説明変数は、適宜、充⾜・削除・作成して頂いて構いません。(2020年のデータは除き、スワローズ以外のチームにして下さい)

# 問題設定
- 予測するチーム：`横浜DeNAベイスターズ`
- 目的変数：ホームゲームの観客数
- 説明変数
    - 試合日程 (年,日付)
    - 勝敗
    - スコア
    - 対戦相手
    - 先発投手
- 使用するモデル：//TODO:後で埋める

- データ元URL：https://baseball-freak.com/audience/23/baystars.html

In [159]:
from typing import Final as const
from IPython.display import display, Markdown
import pandas as pd
from pandas.core.series import Series as Row #TypeHint用
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score
from sklearn.preprocessing import OneHotEncoder
import joblib

from tqdm import tqdm
import re


In [160]:
# インポートするデータの条件指定
data_year_range: const[tuple] = (9, 23)
team: const[str] = "baystars"
base_url: const[str] = "https://baseball-freak.com/audience/"

In [161]:
# 指定範囲年のデータのインポート
datas: list[pd.DataFrame] = []
for year in tqdm(range(data_year_range[0], data_year_range[1]+1), desc="Fetching Data...."):
    url = f"{base_url}{year:02}/{team}.html"
    dfs = pd.read_html(url)
    datas.append(dfs[2])
else:
    print("Done")


Fetching Data....: 100%|██████████| 15/15 [00:04<00:00,  3.32it/s]

Done


In [162]:
# 年数のカラムを追加し、DataFrameを１つにまとめる。
for df, year in zip(datas,  np.arange(data_year_range[0], data_year_range[1]+1).tolist()):
    df['年'] = year

dataset = pd.concat(datas, ignore_index=True)
dataset

,日付,観客数,勝敗,スコア,対戦相手,先発投手,試合時間,球場,日付.1,年
0,4月7日(火),"20,168 人",●,1 - 5,巨人,寺原,3:45,横浜,4月7日(火),9
1,4月8日(水),"16,361 人",●,1 - 12,巨人,工藤,3:10,横浜,4月8日(水),9
2,4月9日(木),"16,691 人",●,2 - 9,巨人,ウォーランド,3:15,横浜,4月9日(木),9
3,4月10日(金),"12,791 人",○,9 - 1,ヤクルト,三浦,2:46,横浜,4月10日(金),9
4,4月11日(土),"17,817 人",●,0 - 3,ヤクルト,グリン,3:23,横浜,4月11日(土),9
...,...,...,...,...,...,...,...,...,...,...
1127,9月25日(月),"33,271 人",○,1 - 0,巨人,大貫,3:13,横浜,9月25日(月),23
1128,9月26日(火),"33,262 人",○,1 - 0,巨人,東,2:31,横浜,9月26日(火),23
1129,9月27日(水),"33,254 人",●,3 - 11,ヤクルト,坂本,3:02,横浜,9月27日(水),23
1130,9月29日(金),"33,267 人",○,5 - 3,阪神,石田,4:00,横浜,9月29日(金),23


In [163]:
# 欠損値のある行は除外する。
dataset.replace('-', np.nan, inplace=True)
dataset.replace('中止', np.nan, inplace=True)
dataset.dropna(inplace=True)
dataset.notna().any()

日付      True
観客数     True
勝敗      True
スコア     True
対戦相手    True
先発投手    True
試合時間    True
球場      True
日付.1    True
年       True
dtype: bool

In [164]:
# 説明変数のみを残す
dataset.drop(columns=["球場", "日付.1", "試合時間",], inplace=True)
dataset


,日付,観客数,勝敗,スコア,対戦相手,先発投手,年
0,4月7日(火),"20,168 人",●,1 - 5,巨人,寺原,9
1,4月8日(水),"16,361 人",●,1 - 12,巨人,工藤,9
2,4月9日(木),"16,691 人",●,2 - 9,巨人,ウォーランド,9
3,4月10日(金),"12,791 人",○,9 - 1,ヤクルト,三浦,9
4,4月11日(土),"17,817 人",●,0 - 3,ヤクルト,グリン,9
...,...,...,...,...,...,...,...
1127,9月25日(月),"33,271 人",○,1 - 0,巨人,大貫,23
1128,9月26日(火),"33,262 人",○,1 - 0,巨人,東,23
1129,9月27日(水),"33,254 人",●,3 - 11,ヤクルト,坂本,23
1130,9月29日(金),"33,267 人",○,5 - 3,阪神,石田,23


In [165]:
# データの成形・前処理

# 観客数の単位（人）をのぞく
def remove_unit(value: str):
    return int(value[:-1].replace(',', ''))

dataset['観客数'] = dataset['観客数'].apply(remove_unit)

# スコアを自軍スコアと相手スコアに分ける
dataset[['自軍スコア', '相手スコア']] = dataset['スコア'].str.split('-', expand=True)
dataset["自軍スコア"] = dataset['自軍スコア'].astype(int)
dataset['相手スコア'] = dataset['相手スコア'].astype(int)
dataset.drop(columns=["スコア"], inplace=True)

#勝敗：勝ち（○）= true, 負け（●） = falseとしてエンコーディングする
dataset["勝利"] = dataset['勝敗'].apply(lambda v: True if v == '○' else False)
dataset.drop(columns=["勝敗"], inplace=True)

# 対戦相手と先発投手をonehotエンコーディングする
df_encoded = pd.get_dummies(dataset, columns=["対戦相手", "先発投手"])

# 日付データの成形
df_encoded['年'] = df_encoded['年'].apply(lambda v: 2000 + int(v))
def convine_date_year(row: Row):
    date_str = row['日付']
    year = row['年']
   
    match = re.match(r'(\d+)月(\d+)日', date_str)
    if match:
        month = int(match.group(1))
        day = int(match.group(2))
        return pd.Timestamp(year=year, month=month, day=day)
    else:
        return None 

df_encoded['年月日'] = df_encoded.apply(convine_date_year, axis=1)
df_encoded.drop(columns=['日付', '年'], inplace=True)
df_encoded['年月日'] = pd.to_datetime(df_encoded['年月日'], format='%Y-%m-%d')
#年, 月, 日データでそれぞれカラムを用意する
df_encoded['年'] = df_encoded['年月日'].dt.year
df_encoded['月'] = df_encoded['年月日'].dt.month
df_encoded['日'] = df_encoded['年月日'].dt.day
df_encoded['曜日'] = df_encoded['年月日'].dt.dayofweek
df_encoded['四半期'] = df_encoded['年月日'].dt.quarter
df_encoded.drop(columns=["年月日"], inplace=True)


df_encoded


,観客数,自軍スコア,相手スコア,勝利,対戦相手_オリックス,対戦相手_ソフトバンク,対戦相手_ヤクルト,対戦相手_ロッテ,対戦相手_中日,対戦相手_巨人,...,先発投手_阿斗里,先発投手_須田,先発投手_飯塚,先発投手_高崎,先発投手_高橋,年,月,日,曜日,四半期
0,20168,1,5,False,False,False,False,False,False,True,...,False,False,False,False,False,2009,4,7,1,2
1,16361,1,12,False,False,False,False,False,False,True,...,False,False,False,False,False,2009,4,8,2,2
2,16691,2,9,False,False,False,False,False,False,True,...,False,False,False,False,False,2009,4,9,3,2
3,12791,9,1,True,False,False,True,False,False,False,...,False,False,False,False,False,2009,4,10,4,2
4,17817,0,3,False,False,False,True,False,False,False,...,False,False,False,False,False,2009,4,11,5,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1127,33271,1,0,True,False,False,False,False,False,True,...,False,False,False,False,False,2023,9,25,0,3
1128,33262,1,0,True,False,False,False,False,False,True,...,False,False,False,False,False,2023,9,26,1,3
1129,33254,3,11,False,False,False,True,False,False,False,...,False,False,False,False,False,2023,9,27,2,3
1130,33267,5,3,True,False,False,False,False,False,False,...,False,False,False,False,False,2023,9,29,4,3


In [166]:
# データセットの分割
target = "観客数"
X = df_encoded.drop(columns=[target])
y = df_encoded[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [167]:
# モデルの構築
estimator = DecisionTreeRegressor(random_state=515)
grid_params = {
    'max_depth': [2, 4, 6, 8, 10, 12],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4, 6, 8, 10, 12],
}

models = GridSearchCV(estimator=estimator, param_grid=grid_params, cv=5, scoring='r2', n_jobs=-1)
models.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=DecisionTreeRegressor(random_state=515), n_jobs=-1,
             param_grid={'max_depth': [2, 4, 6, 8, 10, 12],
                         'min_samples_leaf': [1, 2, 4, 6, 8, 10, 12],
                         'min_samples_split': [2, 5, 10]},
             scoring='r2')

In [168]:
# 最適モデルの抽出
best_model = models.best_estimator_
print(f"model param: {models.best_params_}")
print(f"model learning score: {models.best_score_}")

# 評価
y_pred = best_model.predict(X_test)

r2_value = r2_score(y_test, y_pred)
print(f"TEST R2: {r2_value}")


model param: {'max_depth': 6, 'min_samples_leaf': 10, 'min_samples_split': 2}
model learning score: 0.7758761950833231
TEST R2: 0.7511420633807329


In [169]:
#モデルの保存
joblib.dump(best_model, './model.joblib')
print("Save model.")

Save model.
